In [1]:
import numpy as np
import pandas as pd
import warnings
import os

import sys
sys.path.append('/workspaces/expenses')
import src.utils
import src.prepare_data

warnings.filterwarnings('ignore')
pd.options.display.float_format = "{:,.5f}".format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
config = src.utils.load_config()
datasets = src.utils.load_data(config)
df = src.prepare_data.prepare(datasets)

The config.yml file read was successful.


In [3]:
# vson_members = pd.read_csv("/workspaces/expenses/data/vson_members.csv")
# vson_members.head()

In [3]:
df[df['expense_category'].isnull()].head()

,date,transaction_amount,merchant,file_name,transaction_type,expense_category
4,2023-06-02,-24.00000,GUMDOCTORS PARTNERSHIP EAST GOSFORD,CSVData_credit_outgoing.csv,expense,NaN
11,2023-02-02,-360.00000,SYDNEY ACADEMY OF CHES BURWOOD,CSVData_credit_outgoing.csv,expense,NaN
13,2023-01-31,-16.12000,FACEBK *TGJFSLPBF2 fb.me/ads IRL ## IR...,CSVData_credit_outgoing.csv,expense,NaN
16,2023-01-29,-61.94000,SOUTHERN GROUP PTY L WYONG NSW,CSVData_credit_outgoing.csv,expense,NaN
17,2023-01-28,-19.00000,EB GAMES ERINA,CSVData_credit_outgoing.csv,expense,NaN


In [6]:
df[df['merchant'].str.contains(r'20022315')]

,date,transaction_amount,merchant,file_name,transaction_type,expense_category
274,2022-10-16,"-1,508.40000",C.C.A.S. NetBank BPAY 94482 20022315 CCAS,CSVData_saving_outgoing.csv,expense,kids education
379,2022-09-16,"-1,508.40000",C.C.A.S. NetBank BPAY 94482 20022315 CCAS,CSVData_saving_outgoing.csv,expense,kids education
431,2022-08-16,"-1,508.40000",C.C.A.S. NetBank BPAY 94482 20022315 CCAS,CSVData_saving_outgoing.csv,expense,kids education
514,2022-07-16,"-1,508.40000",C.C.A.S. NetBank BPAY 94482 20022315 CCAS,CSVData_saving_outgoing.csv,expense,kids education
546,2022-06-16,"-1,508.40000",C.C.A.S. NetBank BPAY 94482 20022315 CCAS,CSVData_saving_outgoing.csv,expense,kids education
609,2022-05-16,"-1,508.40000",C.C.A.S. NetBank BPAY 94482 20022315 CCAS,CSVData_saving_outgoing.csv,expense,kids education
716,2022-04-16,"-1,508.40000",C.C.A.S. NetBank BPAY 94482 20022315 CCAS,CSVData_saving_outgoing.csv,expense,kids education
794,2022-03-16,"-1,508.40000",C.C.A.S. NetBank BPAY 94482 20022315 CCAS,CSVData_saving_outgoing.csv,expense,kids education
878,2022-02-16,"-1,508.40000",C.C.A.S. NetBank BPAY 94482 20022315 CCAS,CSVData_saving_outgoing.csv,expense,kids education


In [10]:
df.shape

(990, 6)

In [9]:
calc = (df
          .groupby(['expense_category', 'merchant'])
          .agg(count=('merchant', 'count')
            , sum=('transaction_amount', 'sum')
            , date_from=('date', 'min')
            , date_to=('date', 'max')
                    )
          .reset_index()
)
calc.head(2)

,expense_category,merchant,count,sum,date_from,date_to
0,AGL - Electricity,AGL SALES 0308 MELBOURNE VIC,3,"-1,099.30000",2022-02-12,2022-07-06
1,AGL - GAS,AGL RETAIL ENERGY LIM SYDNEY,1,-39.85000,2022-02-15,2022-02-15


In [7]:
calc2 = (df[df['expense_category'].isnull()]
          .groupby(['merchant'])
          .agg(count=('merchant', 'count')
            , sum=('transaction_amount', 'sum')
            , date_from=('date', 'min')
            , date_to=('date', 'max')
                    )
          .reset_index()
)
calc2.sort_values(by=['count'], ascending=[False]).head()

,merchant,count,sum,date_from,date_to
228,Transfer Fee,7,-42.00000,2022-05-08,2022-12-27
199,SUTCLIFFE & MORT PTY ERINA AU Card xx4289,7,-184.45000,2022-01-29,2023-01-27
2,ALDI STORES - POINT CL POINT CLARE AU,4,-661.27000,2022-03-13,2022-10-24
182,SPOTLIGHT 024 WEST GOSFORD AU,4,-115.74000,2022-04-30,2022-10-04
12,ANACONDA WEST GO WEST GOSFORD AU,4,-539.06000,2022-04-16,2022-10-04


In [5]:
calc.sort_values(by=['count'], ascending=[False])

,expense_category,count,sum,date_from,date_to
11,grocery,212,"-11,656.06000",2022-01-10,2022-12-29
7,eat out,61,"-1,489.03000",2022-01-05,2022-12-30
22,stationary,36,"-1,545.73000",2022-01-10,2022-12-24
4,beverage,33,"-1,353.14000",2022-02-09,2022-12-24
15,insurance,32,"-2,688.09000",2022-02-22,2022-09-23
9,fuel,32,"-2,758.33000",2022-01-07,2022-12-29
10,gardening,29,"-1,513.84000",2022-02-07,2022-12-29
20,professional membership - kb,23,-532.80000,2022-01-04,2022-12-23
6,clothing,22,"-1,788.92000",2022-01-11,2022-11-29
18,life insurance,22,"-4,447.23000",2022-02-15,2022-12-18


In [18]:
df[df['expense_category']=='beverage'].sort_values(by=['date'], ascending=[True]).head()

,date,transaction_amount,merchant,file_name,transaction_type,expense_category
248,2022-02-09,-14.00000,LIQUORLAND 3505 KARIONG AUS,credit.csv,expense,beverage
295,2022-05-10,-120.94000,DAN MURPHY'S 1839 ERINA NSW AU Cash Out $50.00...,saving.csv,expense,beverage
440,2022-05-28,-77.93000,DAN MURPHY'S 1928 WEST GOSFORDAUS,credit.csv,expense,beverage
598,2022-06-14,-19.98000,DAN MURPHY'S 1928 WEST GOSFORD AU AUS Card xx4...,saving.csv,expense,beverage
404,2022-06-17,-43.93000,DAN MURPHY'S 1928 WEST GOSFORDAUS,credit.csv,expense,beverage


In [24]:
df[df['file_name']=='saving.csv']

,date,transaction_amount,merchant,file_name,transaction_type,expense_category
0,2022-12-30,-5.70000,MCDONALDS GOSFORD WST WEST GOSFORD AUS Card xx...,saving.csv,expense,eat out
1,2022-12-30,-2.55000,MCDONALDS GOSFORD WST WEST GOSFORD AUS Card xx...,saving.csv,expense,eat out
2,2022-12-30,-34.76000,MEMET ARSLAN MANLY AUS Card xx4263 Value Date:...,saving.csv,expense,eat out
3,2022-12-30,-21.60000,ROYAL COPENHAGEN CIR SYDNEY AUS Card xx4263 Va...,saving.csv,expense,eat out
4,2022-12-29,-60.00000,ACUPUNCTURE MASSAG ERINA FAIR NS AUS Card xx42...,saving.csv,expense,health & welbeing
5,2022-12-29,-5.20000,WOOLWORTHS 1111 TUGGERAH AU AUS Card xx4263 Va...,saving.csv,expense,grocery
6,2022-12-29,-65.43000,COLES 0821 WEST GOSFORD AU AUS Card xx4263 Val...,saving.csv,expense,grocery
7,2022-12-29,-13.50000,AMPOL KARIONG 28650F KARIONG AUS Card xx4263 V...,saving.csv,expense,NaN
8,2022-12-29,-80.38000,BUNNINGS 569000 TUGGERAH AUS Card xx4263 Value...,saving.csv,expense,gardening
9,2022-12-29,-25.90000,EDO SUSHI & BBQ REST MANLY NS AUS Card xx4263 ...,saving.csv,expense,eat out
